# LIMPANDO_TWITTER 
 - Naive Bayes 

## Análise de Sentimentos

In [1]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import os
import chardet

In [2]:
# getcwd = descobrindo qual diretório está fixado, lembrar que o formato é com  C:\\nome\\nome\\  
# chfdir = para indicar o diretório de trabalho ...

os.chdir('C:\\Users\\particular\\SERIQUE_TCC\\CAPTURA TWITTERS\\')
os.getcwd()

'C:\\Users\\particular\\SERIQUE_TCC\\CAPTURA TWITTERS'

#### TRATA_DADOS
- Junta todos os arquivos do tipo *.csv que existirem no diretório \DADOS para dentro de um único arquivo
- Limpa caracteres desnecessários, deleta colunas desnecessárias

In [3]:
import glob
import os
import pandas as pd

cabecalho = ['Usuario','Texto','Data']

folder_name = 'DADOS'
file_type = 'csv'
separator =';'

dataset = pd.concat([pd.read_csv(f, sep=separator, header=None, usecols=[0,1,2], names=cabecalho) for f in glob.glob(folder_name + "/*."+file_type)], ignore_index=True, axis=0)
#dataset = pd.concat([pd.read_csv(f, sep=separator, header=None, usecols=[0,1,2], names=cabecalho) for f in glob.glob(folder_name + "/*."+file_type)], ignore_index=True)

dataset.to_csv('total_arquivos.csv', sep=separator)
dataset.head()



,Usuario,Texto,Data
0,eudavireis,RT @MaraTellesReal: Austrália - o suicídio de ...,2020-01-10 15:54:23
1,Agu58414164,#Navarro2023 recién dijeron que lo inducieron ...,2020-01-10 15:54:23
2,JuncoWelsh,RT @JCuenta1000: @Andguel SI ! LO FUE ....! (...,2020-01-10 15:54:19
3,rockertux,RT @aaahcarol: vamos falar sobre saúde mental?...,2020-01-10 15:54:16
4,kasouzasz,RT @Larissathielly3: Sextou cm S de suicídio,2020-01-10 15:54:08


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
Usuario    500 non-null object
Texto      500 non-null object
Data       500 non-null object
dtypes: object(3)
memory usage: 11.8+ KB


In [5]:
# Expande o tamanho de exibição de coluna de texto
pd.set_option('display.max_colwidth', -1)
dataset.Texto.head(100)

0     RT @MaraTellesReal: Austrália - o suicídio de um país financiado pela indústria do carvão que elege políticos ultra-conservadores, marketei…                                                                                                                                          
1     #Navarro2023 recién dijeron que lo inducieron al suicidio y ahora esto https://t.co/3HixhvudbC                                                                                                                                                                                        
2     RT @JCuenta1000: @Andguel SI !  LO FUE ....! (ASESINATO)  OJO !!!  ESTÁN INSTALANDO, QUE FUE ASESINADO, PERO POR "OTROS" !!!  COMO QUE LO H…                                                                                                                                          
3     RT @aaahcarol: vamos falar sobre saúde mental?  conversei com meu psiquiatra essa semana e ele me disse que a época do ano com maior índice

**Excluir Linhas duplicadas**

In [6]:
dataset.drop_duplicates(['Texto'],inplace=True)

In [7]:
dataset.count()

Usuario    122
Texto      122
Data       122
dtype: int64

In [8]:
# Visualizando o dataset
dataset.columns

Index(['Usuario', 'Texto', 'Data'], dtype='object')

## Pre-Processamento dos Dados

* Remove linhas duplicadas na base de dados
    - Problema na coleta dos dados.
* Remove Stopwords
* Faz Stemming nos dados
* Remove caracteres indesejados como links, pontuação etc.

In [9]:
tweets = dataset['Texto']


In [10]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\particular\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\particular\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

**Funções de Pre-processamento de dados**

In [11]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [12]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [13]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

In [14]:
import emoji
import regex

def Limpa_emojis(linha):
    emoji_counter = 0
    data = regex.findall(r'\X', linha)
    for word in data:        
        if any(char in emoji.UNICODE_EMOJI for char in word):
            # Remove emoji do texto
            linha = linha.replace(word, '') 
    return (linha)

** Aplica as 3 funções de Pre-processamento nos dados**

In [15]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = Limpa_emojis(instancia)
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('[','').replace(']','').replace('@','').replace('(','').replace('/','')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('#','').replace('RT @','').replace('rt cvvoficial','')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('Á','A').replace('É','E').replace('Í','I').replace('Ó','O').replace('Ú','U')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('!','').replace('?','').replace('ã','a').replace('Ã','A').replace('õ','o').replace('Õ','O')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('â','a').replace('Â','A').replace('ê','e').replace('Ê','E').replace('ô','o').replace('Ô','O')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('💛','').replace('🎗','').replace('🤔','').replace('💡','').replace('☠','').replace('😋','')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('😭','').replace('📼','').replace('🚿','').replace('☢️','').replace('😬','').replace('🙄','')
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('👇','').replace('🔗','').replace('📰','').replace('🖤','').replace('🙌','').replace('🙄','')

    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i)  for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))
 

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

** imprimindo 3 primeiras linhas (antes/depois)para conferir como ficou  ***

In [16]:
print('==> DEPOIS DO PRE PROCESSAMENTO')
tweets[:3]

==> DEPOIS DO PRE PROCESSAMENTO


['rt maratellesre austral suicidi pal financi industr carva eleg poli ultraconservadores, marketei…',
 'navarro2023 recien dijeron lo inducieron al suicidi y ahor est',
 'rt jcuenta1000 andguel si lo fue asesinat ojo estan instalando, fue asesinado, per "otros" lo h…']

In [17]:
# Expande o tamanho de exibição de coluna de texto
print('==> ANTES  DO PRE PROCESSAMENTO')
pd.set_option('display.max_colwidth', -1)
dataset.Texto.head(3)

==> ANTES  DO PRE PROCESSAMENTO


0    RT @MaraTellesReal: Austrália - o suicídio de um país financiado pela indústria do carvão que elege políticos ultra-conservadores, marketei…
1    #Navarro2023 recién dijeron que lo inducieron al suicidio y ahora esto https://t.co/3HixhvudbC                                              
2    RT @JCuenta1000: @Andguel SI !  LO FUE ....! (ASESINATO)  OJO !!!  ESTÁN INSTALANDO, QUE FUE ASESINADO, PERO POR "OTROS" !!!  COMO QUE LO H…
Name: Texto, dtype: object

*** Gravando arquivo para uso na Classificação ***

In [18]:
dataset['Texto_Limpo'] = tweets

In [19]:
dataset.head()

,Usuario,Texto,Data,Texto_Limpo
0,eudavireis,"RT @MaraTellesReal: Austrália - o suicídio de um país financiado pela indústria do carvão que elege políticos ultra-conservadores, marketei…",2020-01-10 15:54:23,"rt maratellesre austral suicidi pal financi industr carva eleg poli ultraconservadores, marketei…"
1,Agu58414164,#Navarro2023 recién dijeron que lo inducieron al suicidio y ahora esto https://t.co/3HixhvudbC,2020-01-10 15:54:23,navarro2023 recien dijeron lo inducieron al suicidi y ahor est
2,JuncoWelsh,"RT @JCuenta1000: @Andguel SI ! LO FUE ....! (ASESINATO) OJO !!! ESTÁN INSTALANDO, QUE FUE ASESINADO, PERO POR ""OTROS"" !!! COMO QUE LO H…",2020-01-10 15:54:19,"rt jcuenta1000 andguel si lo fue asesinat ojo estan instalando, fue asesinado, per ""otros"" lo h…"
3,rockertux,RT @aaahcarol: vamos falar sobre saúde mental? conversei com meu psiquiatra essa semana e ele me disse que a época do ano com maior índice…,2020-01-10 15:54:16,rt aaahcarol vam fal sobr saud ment convers psiquiatr seman diss epoc ano mai indice…
4,kasouzasz,RT @Larissathielly3: Sextou cm S de suicídio,2020-01-10 15:54:08,rt larissathielly3 sext cm s suicidi


In [20]:
dataset.drop('Texto', inplace=True, axis=1)
dataset.drop('Data', inplace=True, axis=1)
dataset.head()

,Usuario,Texto_Limpo
0,eudavireis,"rt maratellesre austral suicidi pal financi industr carva eleg poli ultraconservadores, marketei…"
1,Agu58414164,navarro2023 recien dijeron lo inducieron al suicidi y ahor est
2,JuncoWelsh,"rt jcuenta1000 andguel si lo fue asesinat ojo estan instalando, fue asesinado, per ""otros"" lo h…"
3,rockertux,rt aaahcarol vam fal sobr saud ment convers psiquiatr seman diss epoc ano mai indice…
4,kasouzasz,rt larissathielly3 sext cm s suicidi


In [21]:
os.chdir('C:\\Users\\particular\\SERIQUE_TCC\\')
separator = ';'
dataset.to_csv('total_arquivos_limpo.csv', sep=separator)